In [0]:
import numpy
import os
import librosa
import tensorflow as tf

In [0]:
train_set = numpy.load('train_data.npz')
x_train = train_set['arr_0']
y_train = train_set['arr_1']

In [0]:
test_set = numpy.load('test_data.npz')
x_test = test_set['arr_0']
y_test = test_set['arr_1']

In [0]:
def init_parallel_crnn(input_shape):  
  n_classes = 141
  conv_kernel_size = (3, 1)
  lstm_hidden_units = 64
  # Input layer
  input_layer = tf.keras.layers.Input(input_shape, name='input')
  # Convolutional branch
  # convolution 1
  conv_1 = tf.keras.layers.Conv2D(16, conv_kernel_size, activation='relu')(input_layer)
  max_pool_1 = tf.keras.layers.MaxPooling2D(2, 2)(conv_1)
  # convolution 2
  conv_2 = tf.keras.layers.Conv2D(32, conv_kernel_size, activation='relu')(max_pool_1)
  max_pool_2 = tf.keras.layers.MaxPooling2D(2,2)(conv_2)
  # dropout_1 = tf.keras.layers.Dropout(0.5)(max_pool_2)
  # convolution 3
  conv_3 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_2)
  max_pool_3 = tf.keras.layers.MaxPooling2D(2,2)(conv_3)
  # convolution 4
  conv_4 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_3)
  max_pool_4 = tf.keras.layers.MaxPooling2D(4,4)(conv_4)
  # dropout_2 = tf.keras.layers.Dropout(0.5)(max_pool_4)
  conv_5 = tf.keras.layers.Conv2D(64, conv_kernel_size, activation='relu')(max_pool_4)
  max_pool_5 = tf.keras.layers.MaxPooling2D((4, 4))(conv_5)
  # Flatten
  flatten_1 = tf.keras.layers.Flatten()(max_pool_5)
  # Recurrent branch
  max_pool_lstm = tf.keras.layers.MaxPooling2D(2, 2)(input_layer)
  squeeze_lambda = tf.keras.layers.Lambda(lambda x: tf.keras.backend.squeeze(x, axis= -1))(max_pool_lstm)
  lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_hidden_units))(squeeze_lambda)
  # Concatenate convolutional and recurrent branches
  combine_cnn_rnn = tf.keras.layers.concatenate([flatten_1, lstm], axis=-1, name ='combine')
  # dropout_3 = tf.keras.layers.Dropout(0.5)(combine_cnn_rnn)
  output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')(combine_cnn_rnn)
  model = tf.keras.models.Model(input_layer, output_layer)
  
  print(model.summary())
  return model

In [0]:
def init_crnn(input_shape):  
  n_classes = 141
  n_hidden = 200
  conv_kernel_size = (3, 1)
  lstm_hidden_units = 96
  num_conv_layers = 6
  layers = []
  layers.append(tf.keras.layers.Input(input_shape, name='input'))
  for i in range(num_conv_layers):
    layers.append(tf.keras.layers.Conv1D(filters=56, kernel_size=5, kernel_regularizer=tf.keras.regularizers.l2(0.001), name='conv_'+str(i)))
    layers.append(tf.keras.layers.BatchNormalization(momentum=0.9, name='bNorm_'+str(i)))
    layers.append(tf.keras.layers.Activation('relu', name='act_'+str(i)))
    layers.append(tf.keras.layers.MaxPooling1D(2, name='max_pool_'+str(i)))
  layers.append(tf.keras.layers.LSTM(lstm_hidden_units, return_sequences=False))
  layers.append(tf.keras.layers.Dropout(0.2, name='dropout_1'))
  layers.append(tf.keras.layers.Dense(n_classes*4, activation='relu', name='dense_1'))
  layers.append(tf.keras.layers.Dropout(0.3, name='dropout_2'))
  layers.append(tf.keras.layers.Dense(n_classes*2, activation='relu', name='dense_2'))
  layers.append(tf.keras.layers.Dropout(0.3, name='dropout_3'))
  layers.append(tf.keras.layers.Dense(n_classes, activation='softmax', name='dense_3'))
  model = tf.keras.models.Sequential(layers)
  print(model.summary())
  return model

In [0]:
def init_simple_rnn(input_shape):  
  n_classes = 141
  n_hidden = 200
  model = tf.keras.models.Sequential([tf.keras.layers.LSTM(256, input_shape=input_shape, return_sequences=False),
                                     tf.keras.layers.Dropout(0.2),
                                     tf.keras.layers.Dense(n_hidden, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation='relu'),
                                     tf.keras.layers.Dropout(0.2),
                                     tf.keras.layers.Dense(n_classes, activation='softmax')])  
  
  print(model.summary())
  return model

In [0]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,verbose=1)

callbacks_list = [reducelr_callback]

In [0]:
def build_crnn(x_train, y_train, x_test, y_test, callbacks):
  n_features = x_train.shape[2]
  n_time_frames = x_train.shape[1]
  opt = tf.keras.optimizers.RMSprop(lr=0.0005)  # Optimizer
  model = init_crnn((n_time_frames, n_features))
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  history = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=2, validation_split=0.2, callbacks=callbacks)
  results = model.evaluate(x_test, y_test, batch_size=128)
  print('test loss, test acc:', results)

In [0]:
build_crnn(x_train_set, y_train_set, x_test, y_test, callbacks_list)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_0 (Conv1D)              (None, 858, 56)           35896     
_________________________________________________________________
bNorm_0 (BatchNormalization) (None, 858, 56)           224       
_________________________________________________________________
act_0 (Activation)           (None, 858, 56)           0         
_________________________________________________________________
max_pool_0 (MaxPooling1D)    (None, 429, 56)           0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 425, 56)           15736     
_________________________________________________________________
bNorm_1 (BatchNormalization) (None, 425, 56)           224       
_________________________________________________________________
act_1 (Activation)           (None, 425, 56)          

In [0]:
def build_rnn(x_train, y_train, x_test, y_test, callbacks):
  n_features = x_train.shape[2]
  n_time_frames = x_train.shape[1]
  opt = tf.keras.optimizers.RMSprop(lr=0.0005)  # Optimizer
  model = init_simple_rnn((n_time_frames, n_features))
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  history = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=2, validation_split=0.2, callbacks=callbacks)
  results = model.evaluate(x_test, y_test, batch_size=128)
  print('test loss, test acc:', results)

In [0]:
build_rnn(x_train, y_train, x_test, y_test, callbacks_list)